Careless responder feature engineering and analysis

In [ ]:
#getting and working with data
import pandas as pd
import numpy as np
import re
import os
from itertools import groupby
import datetime as dt
import scipy as sp
import math

#visualizing results
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
#import yellowbrick as yb

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import silhouette_score

In [ ]:
data_path = 'C:/Users/Schindler/Documents/Schindler_Lab/Data/Clinical projects/TILES/final_data/final_data_complete.pkl'

In [ ]:
#read in csv containing data from all surveys
full_data = pd.read_pickle(data_path)
full_data = pd.DataFrame(data = full_data)
full_data.reset_index(inplace=True, drop=True)

print('Original data shape:\n', full_data.shape, '\n')
#ensure no replicate ID (212 participants in study)
print('Original data unique IDs:\n', full_data['ParticipantID'].unique().shape, '\n')
#ensure no replicate ID (212 participants in study)
print('Original data unique IDs:\n', full_data['MitreID'].unique().shape, '\n')
#how much missing data is there?
print('Original data missing value counts:\n', full_data.isnull().sum(), '\n')
#what is the data type of each column?
print('Original data data types:\n', full_data.info(), '\n')

In [ ]:
full_data['survey_type'].value_counts()

In [ ]:
#should be 71
len(full_data['wave_study_day'].unique())

In [ ]:
#create study date bins
full_data['wave_study_date_bin'] = pd.cut(full_data['wave_study_day'], 5)
full_data['wave_study_date_bin'].value_counts()

In [ ]:
full_data.head()

## CR Functions

In [ ]:
def context_homevsworking_CR(row, location_col, activity_col):
    
    #if at home should not be working
    if (row[location_col] == num_dic['location_home']) & (row[activity_col] == num_dic['activity_work']):
        context_homevsworking = 1
    else:
        context_homevsworking = 0

    return context_homevsworking

In [ ]:
def context_workvsactivities_CR(row, location_col, activity_col):
    
    #if at work should not be playing sports, household activities, civic duties
    if (row[location_col] == num_dic['location_work']) & \
    ((row[activity_col] == num_dic['activity_sports']) | (row[activity_col] == num_dic['activity_household']) | (row[activity_col] == num_dic['activity_civic'])):
        context_workvsactivities = 1
    else:
        context_workvsactivities = 0

    return context_workvsactivities

In [ ]:
def context_workvswork_CR(row, location_col, activity_col):
    
    #if at work should be working
    if (row[location_col] == num_dic['location_work']) & (row[activity_col] != num_dic['activity_work']):
        context_workvswork = 1
    else:
        context_workvswork = 0

    return context_workvswork

In [ ]:
def context_drivevsdrive_CR(row, location_col, activity_col):
    
    #if at vehicle should be driving/travel    
    if (row[location_col] == num_dic['location_vehicle']) & (row[activity_col] != num_dic['activity_drive']):
        context_drivevsdrive = 1
    else:
        context_drivevsdrive = 0

    return context_drivevsdrive

In [ ]:
def mean_diff_old(row, param1, param2, param3, param4, neutral_num):
    
    a = row.loc[param1:param2].dropna().values
    b = row.loc[param3:param4].dropna().values
    
    if (len(a) | len(b)) == 0.0:
        mean_diff = np.nan
        return mean_diff
    try:
        if (np.mean(a) == neutral_num) & (np.mean(b) == neutral_num):
            mean_diff = np.nan
            return mean_diff
    except: 
        mean_diff = np.nan
        return mean_diff
    try:
        mean_diff = np.abs(np.mean(a) -  np.mean(b))
        return mean_diff
    except:
        mean_diff = np.nan
        return mean_diff

    return mean_diff

In [ ]:
def mean_diff(row, param1, param2, param3, param4, num_dict):
    
    a = row.loc[param1:param2].dropna().values
    b = row.loc[param3:param4].dropna().values

    a_conv = np.array([num_dict[int(x)] for x in a])
    b_conv = np.array([num_dict[int(x)] for x in b])
    
    if (len(a_conv) | len(b_conv)) == 0.0:
        mean_diff = np.nan
        return mean_diff
    try:
        mean_diff = np.mean(a_conv) +  np.mean(b_conv)
        return mean_diff
    except: 
        mean_diff = np.nan
        return mean_diff

In [ ]:
def median_diff(row, param1, param2, param3, param4):
    
    a = row.loc[param1:param2].values
    b = row.loc[param3:param4].values
    
    if (len(a) | len(b)) == 0.0:
        median_diff = np.nan
    else:
        try:
            median_diff = np.abs(np.median(a) -  np.median(b))
        except:
            median_diff = np.nan
        
    return median_diff

In [ ]:
def mode_diff(row, param1, param2, param3, param4):
    
    a = row.loc[param1:param2].values
    b = row.loc[param3:param4].values
    
    if (len(a) | len(b)) == 0.0:
        mode_diff = np.nan
    else:
        try:
            mode_diff = np.abs(sp.stats.mode(a)[0][0] -  sp.stats.mode(b)[0][0])
        except:
            mode_diff = np.nan
    
    return mode_diff

In [ ]:
def longstring_CR(row, num_questions):
    #create features related to long string analysis (feature of how long the string is and feature of what the string consisted of)

    groups = groupby(row)
    result = [(label, sum(1 for _ in group)) for label, group in groups]
    max_pair = max(result, key=lambda x:x[1])
    max_string_length = max_pair[1]
    max_answer = max_pair[0]

    if max_string_length == 1:
        max_answer = 0
        
    max_string_length_norm = max_string_length / num_questions
    
    return pd.Series((max_string_length, max_answer, max_string_length_norm))

In [ ]:
def moments_CR(row):
    
    #auc
    auc = np.trapz(row.dropna())
    
    #std
    std = np.std(row.dropna())
    
    #skew
    if std == 0.0:
        skew = 0.0
    else:
        try:
            skew = sp.stats.skew(row.dropna())
        except:
            skew = np.nan
    
    #kurtosis
    if std == 0.0:
        kurt = -3.0
    else:
        try:
            kurt = sp.stats.kurtosis(row.dropna())
        except:
            kurt = np.nan
            
    return pd.Series((auc, std, skew, kurt))

In [ ]:
def moments_seeded_CR(row):
    
    #auc
    auc_seeded = np.trapz(np.append(row.dropna().values, 0.0))
    
    #std
    std_seeded = np.std(np.append(row.dropna().values, 0.0))

    #skew
    if std_seeded == 0.0:
        skew_seeded = 0.0
    else:
        try:
            skew_seeded = sp.stats.skew(np.append(row.dropna().values, 0.0))
        except:
            skew_seeded = np.nan
    
    #kurtosis
    if std_seeded == 0.0:
        kurt_seeded = -3.0
    else:
        try:
            kurt_seeded = sp.stats.kurtosis(np.append(row.dropna().values, 0.0))
        except:
            kurt_seeded = np.nan
            
    return pd.Series((auc_seeded, std_seeded, skew_seeded, kurt_seeded))

In [ ]:
def intake_check_CR(row, param1, param2, param3):
    
    #If intake = yes, then SUM(intake types) must be > 0
    if (row[param1] == 1) & (row.loc[param2:param3].sum() <= 0):
        intake_check = 1
    else:
        intake_check = 0

    return intake_check

## CR feature creation

### notes for CR features for Engage surveys

Context question
- Semantic Antonyms
    - if context1 = home (0), then context2 ≠ work and work related (0)
    - if context1 = work (1), then context2 ≠ leisure sports (4), household activities (7), org/civic (11)
- Semantic Synonyms
    - if context1 = work (1), then context2 most likely work and work related (0)
    - If context1 = vehicle (4), then context2 most likely travel or commute (12)
- Internal consistency
    - if context1 = 5 (other) then should have a write in
    - if context2 = 13 (other) then should have a write in

Longstring
- All questions use same scale (1=not at all, 7=very much), but there are 5 different constructs assessed

Semantic consistency
- Internal consistency (within construct) should be greater than consistency across constructs

Semantic synonyms 
- not applicable 

Semantic antonyms
- Hindrance stressors should be negatively correlated with support 


In [ ]:
#split off completed engage and related columns
engage_only = full_data[(full_data['survey_type'] == 'engage_psycap') & (full_data['completed'] == 1.0)]

print(engage_only.shape)
engage_only['ParticipantID'].unique().shape

In [ ]:
%%time

#context
num_dic = {'location_home': 0, 'location_work': 1, 'location_vehicle': 4,
           'activity_work': 0, 'activity_sports': 4, 'activity_household': 7, 'activity_civic': 11, 'activity_drive': 12}
location_col = 'location_num'
activity_col = 'activity_num'
engage_only['context_homevsworking'] = engage_only.apply(lambda row: context_homevsworking_CR(row, location_col, activity_col), axis=1)
engage_only['context_workvsactivities'] = engage_only.apply(lambda row: context_workvsactivities_CR(row, location_col, activity_col), axis=1)
engage_only['context_workvswork'] = engage_only.apply(lambda row: context_workvswork_CR(row, location_col, activity_col), axis=1)
engage_only['context_drivevsdrive'] = engage_only.apply(lambda row: context_drivevsdrive_CR(row, location_col, activity_col), axis=1)

#support_mgt should be negatively correlated with hindrance_mgt
param1 = 'support_mgt'
param2 = 'support_mgt'
param3 = 'hindrance_mgt'
param4 = 'hindrance_mgt'
num_dict = {1: -1.0, 2: -0.66, 3: -0.33, 4: 0.0, 5: 0.33, 6: 0.66, 7: 1.0}
engage_only['mean_diff_hinderance_vs_support'] = engage_only.apply(lambda row: mean_diff(row, param1, param2, param3, param4, num_dict), axis=1)
#engage_only['median_diff_hinderance_vs_support'] = engage_only.apply(lambda row: median_diff(row, param1, param2, param3, param4), axis=1)
#engage_only['mode_diff_hinderance_vs_support'] = engage_only.apply(lambda row: mode_diff(row, param1, param2, param3, param4), axis=1)

#longstring
param1 = 'engage_3'
param2 = 'engage_29'
num_questions = 27
engage_only[['longstring_count_engage', 'longstring_answer_engage', 'longstring_count_norm_engage']] = engage_only.apply(lambda row: longstring_CR(row.loc[param1:param2], num_questions), axis=1)
engage_only['longstring_mult_engage'] = engage_only['longstring_count_engage'] * engage_only['longstring_answer_engage']
engage_only[['ls_auc_engage', 'ls_std_engage', 'ls_skew_engage', 'ls_kurt_engage']] = engage_only.apply(lambda row: moments_CR(row.loc[param1:param2]), axis=1)
engage_only[['ls_auc_seeded_engage', 'ls_std_seeded_engage', 'ls_skew_seeded_engage', 'ls_kurt_seeded_engage']] = engage_only.apply(lambda row: moments_seeded_CR(row.loc[param1:param2]), axis=1)

#norm aves to compare across surveys
engage_only['mean_diff_ave'] = engage_only['mean_diff_hinderance_vs_support']
engage_only['longstring_count_norm_ave'] = engage_only['longstring_count_norm_engage']

### notes for CR features for Psych Flex

Should have answered every question

Longstring
- Legitimate longstrings of  ≥ 8 are unlikely for response “5”
    - make column with longest string
    - make column with number that longest string consisted of

Semantic consistency
- Legitimate scores of pf_mgt=5 are almost impossible

Semantic antonyms
- Not applicable

Semantic synonyms 
- not applicable 


In [ ]:
#split off completed PF and related columns
psych_flex_only = full_data[(full_data['survey_type'] == 'psych_flex') & (full_data['completed'] == 1.0)]

print(psych_flex_only.shape)
psych_flex_only['ParticipantID'].unique().shape

In [ ]:
#longstring
param1 = 'pf_03'
param2 = 'pf_15'
num_questions = 13
psych_flex_only[['longstring_count_pf', 'longstring_answer_pf', 'longstring_count_norm_pf']] = psych_flex_only.apply(lambda row: longstring_CR(row.loc[param1:param2], num_questions), axis=1)
psych_flex_only['longstring_mult_pf'] = psych_flex_only['longstring_count_pf'] * psych_flex_only['longstring_answer_pf']
psych_flex_only[['ls_auc_pf', 'ls_std_pf', 'ls_skew_pf', 'ls_kurt_pf']] = psych_flex_only.apply(lambda row: moments_CR(row.loc[param1:param2]), axis=1)
psych_flex_only[['ls_auc_seeded_pf', 'ls_std_seeded_pf', 'ls_skew_seeded_pf', 'ls_kurt_seeded_pf']] = psych_flex_only.apply(lambda row: moments_seeded_CR(row.loc[param1:param2]), axis=1)

#norm aves to compare across surveys
psych_flex_only['longstring_count_norm_ave'] = psych_flex_only['longstring_count_norm_pf']

### notes for CR features for Jobs

Context question (context 2 = activity, context 3 = location)
- Semantic Antonyms
    - if context3 = home (1), then context2 ≠ work and work related (1)
    - if context3 = work (2), then context2 ≠ leisure sports (3), household activities (6), org/civic (10)
- Semantic Synonyms
    - if context3 = work (2), then context2 most likely work and work related (1)
    - If context3 = vehicle (5), then context2 most likely travel or commute (11)

Affect/Anxiety/Stress
- Longstrings
    - All questions use same scale
- Semantic antonyms
    - Positive block (pan1-5) should be negatively correlated with negative block (pan6-10)

Task Perfomrance
- Longstrings
    - IRB questions use same scale
    - dalal questions use same scale
- Consistency
    - irb2, irb3, irb4 should be negatively correlated with irb6 and irb7
    - itp1, itp2, itp3 should be negatively correlated with irb6 and irb7
    - dalal1-8 should be negatively correlated with dalal9-dalal16


***Question asked if at work or not, if at work then recieve additional questions, if not at work then survey ends; process each type separately for CRs


In [ ]:
#split off completed (eg at work) job and related columns
jobs_atwork_only = full_data[(full_data['survey_type'] == 'job') & (full_data['work'] == 1)]

print(jobs_atwork_only.shape)
jobs_atwork_only['ParticipantID'].unique().shape

In [ ]:
%%time

#update survey type column to reflect if at work or not
jobs_atwork_only['survey_type'] = 'jobs_atwork'

#context comparisons
num_dic = {'location_home': 1, 'location_work': 2, 'location_vehicle': 5,
           'activity_work': 1, 'activity_sports': 3, 'activity_household': 6, 'activity_civic': 10, 'activity_drive': 11}
location_col = 'context3'
activity_col = 'context2'
jobs_atwork_only['context_homevsworking'] = jobs_atwork_only.apply(lambda row: context_homevsworking_CR(row, location_col, activity_col), axis=1)
jobs_atwork_only['context_workvsactivities'] = jobs_atwork_only.apply(lambda row: context_workvsactivities_CR(row, location_col, activity_col), axis=1)
jobs_atwork_only['context_workvswork'] = jobs_atwork_only.apply(lambda row: context_workvswork_CR(row, location_col, activity_col), axis=1)
jobs_atwork_only['context_drivevsdrive'] = jobs_atwork_only.apply(lambda row: context_drivevsdrive_CR(row, location_col, activity_col), axis=1)


#Positive block (pan1-5) should be negatively correlated with negative block (pan6-10)
param1 = 'pand1'
param2 ='pand5'
param3 = 'pand6'
param4 = 'pand10'
num_dict = {1: -1.0, 2: -0.5, 3: 0.0, 4: 0.5, 5: 1.0}
jobs_atwork_only['mean_diff_affect_posneg'] = jobs_atwork_only.apply(lambda row: mean_diff(row, param1, param2, param3, param4, num_dict), axis=1)
#jobs_atwork_only['median_diff_affect_posneg'] = jobs_atwork_only.apply(lambda row: median_diff(row, param1, param2, param3, param4), axis=1)
#jobs_atwork_only['mode_diff_affect_posneg'] = jobs_atwork_only.apply(lambda row: mode_diff(row, param1, param2, param3, param4), axis=1)
 
#pand10 (nervousness) should be positivitely correlated with anxiety
param1 = 'pand10'
param2 ='pand10'
param3 = 'anxiety'
param4 = 'anxiety'
num_dict = {1: -1.0, 2: -0.5, 3: 0.0, 4: 0.5, 5: 1.0}
jobs_atwork_only['mean_diff_nervous_anxiety'] = jobs_atwork_only.apply(lambda row: mean_diff(row, param1, param2, param3, param4, num_dict), axis=1)
#jobs_atwork_only['median_diff_nervous_anxiety'] = jobs_atwork_only.apply(lambda row: median_diff(row, param1, param2, param3, param4), axis=1)
#jobs_atwork_only['mode_diff_nervous_anxiety'] = jobs_atwork_only.apply(lambda row: mode_diff(row, param1, param2, param3, param4), axis=1)
  
#irb1, irb2, irb3, irb4 should be negatively correlated with irb6 and irb7
param1 = 'irbd1'
param2 ='irbd4'
param3 = 'irbd6'
param4 = 'irbd7'
num_dict = {1: -1.0, 2: -0.66, 3: -0.33, 4: 0.0, 5: 0.33, 6: 0.66, 7: 1.0}
jobs_atwork_only['mean_diff_irb_irb'] = jobs_atwork_only.apply(lambda row: mean_diff(row, param1, param2, param3, param4, num_dict), axis=1)
#jobs_atwork_only['median_diff_irb_irb'] = jobs_atwork_only.apply(lambda row: median_diff(row, param1, param2, param3, param4), axis=1)
#jobs_atwork_only['mode_diff_irb_irb'] = jobs_atwork_only.apply(lambda row: mode_diff(row, param1, param2, param3, param4), axis=1)

#itp1, itp2, itp3 should be negatively correlated with irb6 and irb7
#param1 = 'itpd1'
#param2 ='itpd3'
#param3 = 'irbd6'
#param4 = 'irbd7'
#jobs_atwork_only['mean_diff_irb_itp'] = jobs_atwork_only.apply(lambda row: mean_diff(row, param1, param2, param3, param4), axis=1)
#jobs_atwork_only['median_diff_irb_itp'] = jobs_atwork_only.apply(lambda row: median_diff(row, param1, param2, param3, param4), axis=1)
#jobs_atwork_only['mode_diff_irb_itp'] = jobs_atwork_only.apply(lambda row: mode_diff(row, param1, param2, param3, param4), axis=1)

#dalal1-8 should be negatively correlated with dalal9-dalal16
param1 = 'dalal1'
param2 ='dalal8'
param3 = 'dalal9'
param4 = 'dalal16'
num_dict = {1: -1.0, 2: 1.0}
jobs_atwork_only['mean_diff_dalal_posneg'] = jobs_atwork_only.apply(lambda row: mean_diff(row, param1, param2, param3, param4, num_dict), axis=1)
#jobs_atwork_only['median_diff_dalal_posneg'] = jobs_atwork_only.apply(lambda row: median_diff(row, param1, param2, param3, param4), axis=1)
#jobs_atwork_only['mode_diff_dalal_posneg'] = jobs_atwork_only.apply(lambda row: mode_diff(row, param1, param2, param3, param4), axis=1)

#affect longstring
param1 = 'pand1'
param2 = 'pand10'
num_questions = 10
jobs_atwork_only[['longstring_count_affect', 'longstring_answer_affect', 'longstring_count_norm_affect']] = jobs_atwork_only.apply(lambda row: longstring_CR(row.loc[param1:param2], num_questions), axis=1)
jobs_atwork_only['longstring_mult_affect'] = jobs_atwork_only['longstring_count_affect'] * jobs_atwork_only['longstring_answer_affect']
jobs_atwork_only[['ls_auc_affect', 'ls_std_affect', 'ls_skew_affect', 'ls_kurt_affect']] = jobs_atwork_only.apply(lambda row: moments_CR(row.loc[param1:param2]), axis=1)
jobs_atwork_only[['ls_auc_seeded_affect', 'ls_std_seeded_affect', 'ls_skew_seeded_affect', 'ls_kurt_seeded_affect']] = jobs_atwork_only.apply(lambda row: moments_seeded_CR(row.loc[param1:param2]), axis=1)

#irbd longstring
param1 = 'irbd1'
param2 = 'irbd7'
num_questions = 7
jobs_atwork_only[['longstring_count_irbd', 'longstring_answer_irbd', 'longstring_count_norm_irbd']] = jobs_atwork_only.apply(lambda row: longstring_CR(row.loc[param1:param2], num_questions), axis=1)
jobs_atwork_only['longstring_mult_irbd'] = jobs_atwork_only['longstring_count_irbd'] * jobs_atwork_only['longstring_answer_irbd']
jobs_atwork_only[['ls_auc_irbd', 'ls_std_irbd', 'ls_skew_irbd', 'ls_kurt_irbd']] = jobs_atwork_only.apply(lambda row: moments_CR(row.loc[param1:param2]), axis=1)
jobs_atwork_only[['ls_auc_seeded_irbd', 'ls_std_seeded_irbd', 'ls_skew_seeded_irbd', 'ls_kurt_seeded_irbd']] = jobs_atwork_only.apply(lambda row: moments_seeded_CR(row.loc[param1:param2]), axis=1)

#dalal longstring
param1 = 'dalal1'
param2 = 'dalal16'
num_questions = 16
jobs_atwork_only[['longstring_count_dalal', 'longstring_answer_dalal', 'longstring_count_norm_dalal']] = jobs_atwork_only.apply(lambda row: longstring_CR(row.loc[param1:param2], num_questions), axis=1)
jobs_atwork_only['longstring_mult_dalal'] = jobs_atwork_only['longstring_count_dalal'] * jobs_atwork_only['longstring_answer_dalal']
jobs_atwork_only[['ls_auc_dalal', 'ls_std_dalal', 'ls_skew_dalal', 'ls_kurt_dalal']] = jobs_atwork_only.apply(lambda row: moments_CR(row.loc[param1:param2]), axis=1)
jobs_atwork_only[['ls_auc_seeded_dalal', 'ls_std_seeded_dalal', 'ls_skew_seeded_dalal', 'ls_kurt_seeded_dalal']] = jobs_atwork_only.apply(lambda row: moments_seeded_CR(row.loc[param1:param2]), axis=1)

#norm aves to compare across surveys
jobs_atwork_only['mean_diff_ave'] = jobs_atwork_only[['mean_diff_affect_posneg', 'mean_diff_nervous_anxiety', 'mean_diff_irb_irb', 'mean_diff_dalal_posneg']].mean(axis=1)
jobs_atwork_only['longstring_count_norm_ave'] = jobs_atwork_only[['longstring_count_norm_affect', 'longstring_count_norm_irbd', 'longstring_count_norm_dalal']].mean(axis=1)

In [ ]:
#split off non at work job and related columns
jobs_notatwork_only = full_data[(full_data['survey_type'] == 'job') & (full_data['work'] == 2)]

print(jobs_notatwork_only.shape)
jobs_notatwork_only['ParticipantID'].unique().shape

In [ ]:
%%time

#update survey type column to reflect if at work or not
jobs_notatwork_only['survey_type'] = 'jobs_notatwork'

#context comparisons
num_dic = {'location_home': 1, 'location_work': 2, 'location_vehicle': 5,
           'activity_work': 1, 'activity_sports': 3, 'activity_household': 6, 'activity_civic': 10, 'activity_drive': 11}
location_col = 'context3'
activity_col = 'context2'
jobs_notatwork_only['context_homevsworking'] = jobs_notatwork_only.apply(lambda row: context_homevsworking_CR(row, location_col, activity_col), axis=1)
jobs_notatwork_only['context_workvsactivities'] = jobs_notatwork_only.apply(lambda row: context_workvsactivities_CR(row, location_col, activity_col), axis=1)
jobs_notatwork_only['context_workvswork'] = jobs_notatwork_only.apply(lambda row: context_workvswork_CR(row, location_col, activity_col), axis=1)
jobs_notatwork_only['context_drivevsdrive'] = jobs_notatwork_only.apply(lambda row: context_drivevsdrive_CR(row, location_col, activity_col), axis=1)

#Positive block (pan1-5) should be negatively correlated with negative block (pan6-10)
param1 = 'pand1'
param2 ='pand5'
param3 = 'pand6'
param4 = 'pand10'
num_dict = {1: -1.0, 2: -0.5, 3: 0.0, 4: 0.5, 5: 1.0}
jobs_notatwork_only['mean_diff_affect_posneg'] = jobs_notatwork_only.apply(lambda row: mean_diff(row, param1, param2, param3, param4, num_dict), axis=1)
#jobs_notatwork_only['median_diff_affect_posneg'] = jobs_notatwork_only.apply(lambda row: median_diff(row, param1, param2, param3, param4), axis=1)
#jobs_notatwork_only['mode_diff_affect_posneg'] = jobs_notatwork_only.apply(lambda row: mode_diff(row, param1, param2, param3, param4), axis=1)
 
#pand10 (nervousness) should be positivitely correlated with anxiety
param1 = 'pand10'
param2 ='pand10'
param3 = 'anxiety'
param4 = 'anxiety'
num_dict = {1: -1.0, 2: -0.5, 3: 0.0, 4: 0.5, 5: 1.0}
jobs_notatwork_only['mean_diff_nervous_anxiety'] = jobs_notatwork_only.apply(lambda row: mean_diff(row, param1, param2, param3, param4, num_dict), axis=1)
#jobs_notatwork_only['median_diff_nervous_anxiety'] = jobs_notatwork_only.apply(lambda row: median_diff(row, param1, param2, param3, param4), axis=1)
#jobs_notatwork_only['mode_diff_nervous_anxiety'] = jobs_notatwork_only.apply(lambda row: mode_diff(row, param1, param2, param3, param4), axis=1)

#affect longstring
param1 = 'pand1'
param2 = 'pand10'
num_questions = 10
jobs_notatwork_only[['longstring_count_affect', 'longstring_answer_affect', 'longstring_count_norm_affect']] = jobs_notatwork_only.apply(lambda row: longstring_CR(row.loc[param1:param2], num_questions), axis=1)
jobs_notatwork_only['longstring_mult_affect'] = jobs_notatwork_only['longstring_count_affect'] * jobs_notatwork_only['longstring_answer_affect']
jobs_notatwork_only[['ls_auc_affect', 'ls_std_affect', 'ls_skew_affect', 'ls_kurt_affect']] = jobs_notatwork_only.apply(lambda row: moments_CR(row.loc[param1:param2]), axis=1)
jobs_notatwork_only[['ls_auc_seeded_affect', 'ls_std_seeded_affect', 'ls_skew_seeded_affect', 'ls_kurt_seeded_affect']] = jobs_notatwork_only.apply(lambda row: moments_seeded_CR(row.loc[param1:param2]), axis=1)

#norm aves to compare across surveys
jobs_notatwork_only['mean_diff_ave'] = jobs_notatwork_only[['mean_diff_affect_posneg', 'mean_diff_nervous_anxiety']].mean(axis=1)
jobs_notatwork_only['longstring_count_norm_ave'] = jobs_notatwork_only['longstring_count_norm_affect']

### notes for CR features for Health

Context question (context 2 = activity, context 3 = location)
- Semantic Antonyms
    - if context3 = home (1), then context2 ≠ work and work related (1)
    - if context3 = work (2), then context2 ≠ leisure sports (3), household activities (6), org/civic (10)
- Semantic Synonyms
    - if context3 = work (2), then context2 most likely work and work related (1)
    - If context3 = vehicle (5), then context2 most likely travel or commute (11)

Affect/Anxiety/Stress
- Longstrings
    - All questions use same scale
- Semantic antonyms
    - Positive block (pan1-5) should be negatively correlated with negative block (pan6-10)

Task Perfomrance
- Longstrings
    - n/a
- Consistency
    - If alc1 = yes, then SUM(alc2_1, alc2_2, alc2_3) must be > 0
    - If tob1 = yes, then SUM(tob2_1, tob2_2, tob2_3, tob2_4, tob2_5, tob2_6, tob2_7) must be > 0


In [ ]:
#split off completed job and related columns
health_only = full_data[(full_data['survey_type'] == 'health')]

print(health_only.shape)
health_only['ParticipantID'].unique().shape

In [ ]:
%%time

#context comparisons
num_dic = {'location_home': 1, 'location_work': 2, 'location_vehicle': 5,
           'activity_work': 1, 'activity_sports': 3, 'activity_household': 6, 'activity_civic': 10, 'activity_drive': 11}
location_col = 'context3'
activity_col = 'context2'
health_only['context_homevsworking'] = health_only.apply(lambda row: context_homevsworking_CR(row, location_col, activity_col), axis=1)
health_only['context_workvsactivities'] = health_only.apply(lambda row: context_workvsactivities_CR(row, location_col, activity_col), axis=1)
health_only['context_workvswork'] = health_only.apply(lambda row: context_workvswork_CR(row, location_col, activity_col), axis=1)
health_only['context_drivevsdrive'] = health_only.apply(lambda row: context_drivevsdrive_CR(row, location_col, activity_col), axis=1)

#Positive block (pan1-5) should be negatively correlated with negative block (pan6-10)
param1 = 'pand1'
param2 ='pand5'
param3 = 'pand6'
param4 = 'pand10'
num_dict = {1: -1.0, 2: -0.5, 3: 0.0, 4: 0.5, 5: 1.0}
health_only['mean_diff_affect_posneg'] = health_only.apply(lambda row: mean_diff(row, param1, param2, param3, param4, num_dict), axis=1)
#health_only['median_diff_affect_posneg'] = health_only.apply(lambda row: median_diff(row, param1, param2, param3, param4), axis=1)
#health_only['mode_diff_affect_posneg'] = health_only.apply(lambda row: mode_diff(row, param1, param2, param3, param4), axis=1)
 
#pand10 (nervousness) should be positivitely correlated with anxiety
param1 = 'pand10'
param2 ='pand10'
param3 = 'anxiety'
param4 = 'anxiety'
num_dict = {1: -1.0, 2: -0.5, 3: 0.0, 4: 0.5, 5: 1.0}
health_only['mean_diff_nervous_anxiety'] = health_only.apply(lambda row: mean_diff(row, param1, param2, param3, param4, num_dict), axis=1)
#health_only['median_diff_nervous_anxiety'] = health_only.apply(lambda row: median_diff(row, param1, param2, param3, param4), axis=1)
#health_only['mode_diff_nervous_anxiety'] = health_only.apply(lambda row: mode_diff(row, param1, param2, param3, param4), axis=1)
  
#affect longstring
param1 = 'pand1'
param2 = 'pand10'
num_questions = 10
health_only[['longstring_count_affect', 'longstring_answer_affect', 'longstring_count_norm_affect']] = health_only.apply(lambda row: longstring_CR(row.loc[param1:param2], num_questions), axis=1)
health_only['longstring_mult_affect'] = health_only['longstring_count_affect'] * health_only['longstring_answer_affect']
health_only[['ls_auc_affect', 'ls_std_affect', 'ls_skew_affect', 'ls_kurt_affect']] = health_only.apply(lambda row: moments_CR(row.loc[param1:param2]), axis=1)
health_only[['ls_auc_seeded_affect', 'ls_std_seeded_affect', 'ls_skew_seeded_affect', 'ls_kurt_seeded_affect']] = health_only.apply(lambda row: moments_seeded_CR(row.loc[param1:param2]), axis=1)

#alc consistency
param1 = 'alc1'
param2 = 'alc2_1'
param3 = 'alc2_3'
health_only['intake_check_alc'] = health_only.apply(lambda row: intake_check_CR(row, param1, param2, param3), axis=1)

#nicotine consistency
param1 = 'tob1'
param2 = 'tob2_1'
param3 = 'tob2_7'
health_only['intake_check_nic'] = health_only.apply(lambda row: intake_check_CR(row, param1, param2, param3), axis=1)

#norm aves to compare across surveys
health_only['mean_diff_ave'] = health_only[['mean_diff_affect_posneg', 'mean_diff_nervous_anxiety']].mean(axis=1)
health_only['longstring_count_norm_ave'] = health_only['longstring_count_norm_affect']

### notes for CR features for Personality

Context question (context 2 = activity, context 3 = location)
- Semantic Antonyms
    - if context3 = home (1), then context2 ≠ work and work related (1)
    - if context3 = work (2), then context2 ≠ leisure sports (3), household activities (6), org/civic (10)
- Semantic Synonyms
    - if context3 = work (2), then context2 most likely work and work related (1)
    - If context3 = vehicle (5), then context2 most likely travel or commute (11)

Affect/Anxiety/Stress
- Longstrings
    - All questions use same scale
- Semantic antonyms
    - Positive block (pan1-5) should be negatively correlated with negative block (pan6-10)

Task Perfomrance
- Longstrings
    - All questions use same scale
- Consistency - each pair should be negatively correlated (eg each pair is reverse scored)
    - bfi1 and bfi6 (extraversion)
    - bfi2 and bfi7 (agreeableness)
    - bfi3 and bfi8 (conscientiousness)
    - bfi4 and bfi9 (neuroticism)
    - bfi5 and bfi10 (openness)


In [ ]:
#split off completed job and related columns
personality_only = full_data[(full_data['survey_type'] == 'personality')]

print(personality_only.shape)
personality_only['ParticipantID'].unique().shape

In [ ]:
%%time

#context comparisons
num_dic = {'location_home': 1, 'location_work': 2, 'location_vehicle': 5,
           'activity_work': 1, 'activity_sports': 3, 'activity_household': 6, 'activity_civic': 10, 'activity_drive': 11}
location_col = 'context3'
activity_col = 'context2'
personality_only['context_homevsworking'] = personality_only.apply(lambda row: context_homevsworking_CR(row, location_col, activity_col), axis=1)
personality_only['context_workvsactivities'] = personality_only.apply(lambda row: context_workvsactivities_CR(row, location_col, activity_col), axis=1)
personality_only['context_workvswork'] = personality_only.apply(lambda row: context_workvswork_CR(row, location_col, activity_col), axis=1)
personality_only['context_drivevsdrive'] = personality_only.apply(lambda row: context_drivevsdrive_CR(row, location_col, activity_col), axis=1)

#Positive block (pan1-5) should be negatively correlated with negative block (pan6-10)
param1 = 'pand1'
param2 ='pand5'
param3 = 'pand6'
param4 = 'pand10'
num_dict = {1: -1.0, 2: -0.5, 3: 0.0, 4: 0.5, 5: 1.0}
personality_only['mean_diff_affect_posneg'] = personality_only.apply(lambda row: mean_diff(row, param1, param2, param3, param4, num_dict), axis=1)
#personality_only['median_diff_affect_posneg'] = personality_only.apply(lambda row: median_diff(row, param1, param2, param3, param4), axis=1)
#personality_only['mode_diff_affect_posneg'] = personality_only.apply(lambda row: mode_diff(row, param1, param2, param3, param4), axis=1)
 
#pand10 (nervousness) should be positivitely correlated with anxiety
param1 = 'pand10'
param2 ='pand10'
param3 = 'anxiety'
param4 = 'anxiety'
num_dict = {1: -1.0, 2: -0.5, 3: 0.0, 4: 0.5, 5: 1.0}
personality_only['mean_diff_nervous_anxiety'] = personality_only.apply(lambda row: mean_diff(row, param1, param2, param3, param4, num_dict), axis=1)
#personality_only['median_diff_nervous_anxiety'] = personality_only.apply(lambda row: median_diff(row, param1, param2, param3, param4), axis=1)
#personality_only['mode_diff_nervous_anxiety'] = personality_only.apply(lambda row: mode_diff(row, param1, param2, param3, param4), axis=1)
    
#bfi1 and bfi6 (extraversion) should be negatively correlated (eg each pair is reverse scored)
param1 = 'bfid1'
param2 ='bfid1'
param3 = 'bfid6'
param4 = 'bfid6'
num_dict = {1: -1.0, 2: -0.5, 3: 0.0, 4: 0.5, 5: 1.0}
personality_only['mean_diff_extraversion'] = personality_only.apply(lambda row: mean_diff(row, param1, param2, param3, param4, num_dict), axis=1)
#personality_only['median_diff_extraversion'] = personality_only.apply(lambda row: median_diff(row, param1, param2, param3, param4), axis=1)
#personality_only['mode_diff_extraversion'] = personality_only.apply(lambda row: mode_diff(row, param1, param2, param3, param4), axis=1)

#bfi2 and bfi7 (agreeableness) should be negatively correlated (eg each pair is reverse scored)
param1 = 'bfid2'
param2 ='bfid2'
param3 = 'bfid7'
param4 = 'bfid7'
num_dict = {1: -1.0, 2: -0.5, 3: 0.0, 4: 0.5, 5: 1.0}
personality_only['mean_diff_agreeableness'] = personality_only.apply(lambda row: mean_diff(row, param1, param2, param3, param4, num_dict), axis=1)
#personality_only['median_diff_agreeableness'] = personality_only.apply(lambda row: median_diff(row, param1, param2, param3, param4), axis=1)
#personality_only['mode_diff_agreeableness'] = personality_only.apply(lambda row: mode_diff(row, param1, param2, param3, param4), axis=1)

#bfi3 and bfi8 (conscientiousness) should be negatively correlated (eg each pair is reverse scored)
param1 = 'bfid3'
param2 ='bfid3'
param3 = 'bfid8'
param4 = 'bfid8'
num_dict = {1: -1.0, 2: -0.5, 3: 0.0, 4: 0.5, 5: 1.0}
personality_only['mean_diff_conscientiousness'] = personality_only.apply(lambda row: mean_diff(row, param1, param2, param3, param4, num_dict), axis=1)
#personality_only['median_diff_conscientiousness'] = personality_only.apply(lambda row: median_diff(row, param1, param2, param3, param4), axis=1)
#personality_only['mode_diff_conscientiousness'] = personality_only.apply(lambda row: mode_diff(row, param1, param2, param3, param4), axis=1)

#bfi4 and bfi9 (neuroticism) should be negatively correlated (eg each pair is reverse scored)
param1 = 'bfid4'
param2 ='bfid4'
param3 = 'bfid9'
param4 = 'bfid9'
num_dict = {1: -1.0, 2: -0.5, 3: 0.0, 4: 0.5, 5: 1.0}
personality_only['mean_diff_neuroticism'] = personality_only.apply(lambda row: mean_diff(row, param1, param2, param3, param4, num_dict), axis=1)
#personality_only['median_diff_neuroticism'] = personality_only.apply(lambda row: median_diff(row, param1, param2, param3, param4), axis=1)
#personality_only['mode_diff_neuroticism'] = personality_only.apply(lambda row: mode_diff(row, param1, param2, param3, param4), axis=1)

#bfi5 and bfi10 (openness) should be negatively correlated (eg each pair is reverse scored)
param1 = 'bfid5'
param2 ='bfid5'
param3 = 'bfid10'
param4 = 'bfid10'
num_dict = {1: -1.0, 2: -0.5, 3: 0.0, 4: 0.5, 5: 1.0}
personality_only['mean_diff_openness'] = personality_only.apply(lambda row: mean_diff(row, param1, param2, param3, param4, num_dict), axis=1)
#personality_only['median_diff_openness'] = personality_only.apply(lambda row: median_diff(row, param1, param2, param3, param4), axis=1)
#personality_only['mode_diff_openness'] = personality_only.apply(lambda row: mode_diff(row, param1, param2, param3, param4), axis=1)

#affect longstring
param1 = 'pand1'
param2 = 'pand10'
num_questions = 10
personality_only[['longstring_count_affect', 'longstring_answer_affect', 'longstring_count_norm_affect']] = personality_only.apply(lambda row: longstring_CR(row.loc[param1:param2], num_questions), axis=1)
personality_only['longstring_mult_affect'] = personality_only['longstring_count_affect'] * personality_only['longstring_answer_affect']
personality_only[['ls_auc_affect', 'ls_std_affect', 'ls_skew_affect', 'ls_kurt_affect']] = personality_only.apply(lambda row: moments_CR(row.loc[param1:param2]), axis=1)
personality_only[['ls_auc_seeded_affect', 'ls_std_seeded_affect', 'ls_skew_seeded_affect', 'ls_kurt_seeded_affect']] = personality_only.apply(lambda row: moments_seeded_CR(row.loc[param1:param2]), axis=1)

#personality longstring
param1 = 'bfid1'
param2 = 'bfid10'
num_questions = 10
personality_only[['longstring_count_personality', 'longstring_answer_personality', 'longstring_count_norm_personality']] = personality_only.apply(lambda row: longstring_CR(row.loc[param1:param2], num_questions), axis=1)
personality_only['longstring_mult_personality'] = personality_only['longstring_count_personality'] * personality_only['longstring_answer_affect']
personality_only[['ls_auc_personality', 'ls_std_personality', 'ls_skew_personality', 'ls_kurt_personality']] = personality_only.apply(lambda row: moments_CR(row.loc[param1:param2]), axis=1)
personality_only[['ls_auc_seeded_personality', 'ls_std_seeded_personality', 'ls_skew_seeded_personality', 'ls_kurt_seeded_personality']] = personality_only.apply(lambda row: moments_seeded_CR(row.loc[param1:param2]), axis=1)

#norm aves to compare across surveys
personality_only['mean_diff_ave'] = personality_only[['mean_diff_affect_posneg', 'mean_diff_nervous_anxiety', 'mean_diff_extraversion', 'mean_diff_agreeableness', 'mean_diff_conscientiousness', 'mean_diff_neuroticism', 'mean_diff_openness']].mean(axis=1)
personality_only['longstring_count_norm_ave'] = personality_only[['longstring_count_norm_affect', 'longstring_count_norm_personality']].mean(axis=1)

In [ ]:
surveys_CR = pd.concat([engage_only, psych_flex_only, jobs_atwork_only, jobs_notatwork_only, health_only, personality_only], axis = 0, sort=False)
surveys_CR.to_csv('surveys_CR.csv')
surveys_CR.to_pickle('surveys_CR.pkl')
surveys_CR.head()